In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
from lxml import etree

In [2]:
def get_html(url):
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    
    headers = {
        "User-Agent": USER_AGENT
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # 抛出异常如果请求不成功
        html = response.text
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        html = ""
    
    return html

In [3]:
def get_data(html):
    datalist = []
    soup = BeautifulSoup(html, "html.parser")
    items = soup.find_all('li', class_="clearfix")
    for i, item in enumerate(items, start=1):
        data = []
        item = str(item)
        data.append(i)
        gm = re.findall(r'javascript:;">(.*?)</a>', item)[0]
        data.append(gm)
        
        # 修改部分开始
        item_html = etree.HTML(item)
        if i <= 11:
            bfl_text = item_html.xpath('.//p/text()')[0]
        else:
            bfl_text = item_html.xpath('.//text()')[0]
        
        # 提取歌手名和播放量
        match = re.search(r'(\w+)\xa0/\xa0(\d+)次播放', bfl_text)
        if match:
            singer = match.group(1)
            play_count = match.group(2)
            data.append(singer)
            data.append(play_count)
        # 修改部分结束
        
        datalist.append(data)
        if i == 15:
            break
    return datalist

In [4]:
def save_to_excel_txt(datalist, savexlsxpath, savetxtpath):
    book = Workbook()
    sheet = book.active
    sheet.title = '豆瓣音乐本周音乐人最热单曲排行榜'
    col = ("排名", "歌名", "歌手名", "播放量")
    
    # 写入表头
    for i, value in enumerate(col):
        sheet.cell(row=1, column=i+1, value=value)

    # 写入数据
    for i, data in enumerate(datalist):
        data[-1] = data[-1].replace('次播放', '')
        for j, value in enumerate(data):
            sheet.cell(row=i+2, column=j+1, value=value)

    book.save(savexlsxpath)
    wb = load_workbook(savexlsxpath)
    sheet = wb.active
    
    with open(savetxtpath, 'w', encoding='utf-8') as f:
        f.write("排名\t歌名\t歌手名\t播放量\n")
        for data in datalist:
            data[-1] = data[-1].replace('次播放', '')
            line = '\t'.join(str(cell) for cell in data) + '\n'
            f.write(line)

In [5]:
url = "https://music.douban.com/chart"
html = get_html(url)
datalist = get_data(html)

In [6]:
savepath = "data/music.xlsx"
savetxtpath = "data/music_rank.txt"
save_to_excel_txt(datalist, savepath, savetxtpath)
print("爬取完毕！")

爬取完毕！


In [7]:
data = pd.read_excel('data/music.xlsx', na_filter=' ? ')
data

排名                            歌名         歌手名      播放量
0    1                         say88       布布布莱克  56508.0
1    2                  74-见一面吧-1500         张弦织   1318.0
2    3                           四九城           啸  61053.0
3    4            Check It Out Y'All     Post80s  37118.0
4    5  theo's body without organism         贾博昊  36482.0
5    6              而我的青春只有你（流行2000）   博友文化音乐工作室   4947.0
6    7                            古镇         刘海洋  43031.0
7    8      Ain't Got No Where to Go       KurtY  16875.0
8    9                         NEW2      machine  31139.0
9   10                20210711-王子独归路  OTMF一次性交朋友  27813.0
10  11           彭智烨 -【4 Morceaux】送别         NaN      NaN
11  12                   在皑皑白雪上写你的名字         NaN      NaN
12  13                         定稿mix         NaN      NaN
13  14      01 So Right(新乐队DEMO).mp3         NaN      NaN
14  15                        人民需要发声         NaN      NaN

In [8]:
data.sort_values(by='播放量', ascending=False)

排名                            歌名         歌手名      播放量
2    3                           四九城           啸  61053.0
0    1                         say88       布布布莱克  56508.0
6    7                            古镇         刘海洋  43031.0
3    4            Check It Out Y'All     Post80s  37118.0
4    5  theo's body without organism         贾博昊  36482.0
8    9                         NEW2      machine  31139.0
9   10                20210711-王子独归路  OTMF一次性交朋友  27813.0
7    8      Ain't Got No Where to Go       KurtY  16875.0
5    6              而我的青春只有你（流行2000）   博友文化音乐工作室   4947.0
1    2                  74-见一面吧-1500         张弦织   1318.0
10  11           彭智烨 -【4 Morceaux】送别         NaN      NaN
11  12                   在皑皑白雪上写你的名字         NaN      NaN
12  13                         定稿mix         NaN      NaN
13  14      01 So Right(新乐队DEMO).mp3         NaN      NaN
14  15                        人民需要发声         NaN      NaN

In [9]:
data[data['播放量'] > 30000]

排名                            歌名      歌手名      播放量
0   1                         say88    布布布莱克  56508.0
2   3                           四九城        啸  61053.0
3   4            Check It Out Y'All  Post80s  37118.0
4   5  theo's body without organism      贾博昊  36482.0
6   7                            古镇      刘海洋  43031.0
8   9                         NEW2   machine  31139.0